# Optilab tutorial: performing optimization
This tutorial notebook aims to explain how to perform optimization using optilab API. In this notebook you will learn how to:
- create an instance of optimizer and objective function,
- run optimization on the function,
- visualize the results of the optimization,
- save optimization result to a pickle file,
- read and visualize the results using optilab's CLI tool.

## Creating an objective function
Optilab comes with a lot of common black-box optimization functions. Here, lets create instances of a sphere function. Let's say we want to perform optimization in 10 dimensions:

In [ ]:
# define a constant to hold the dimensionality of the problem
DIM = 10

# import the sphere function from optilab
from optilab.functions.unimodal import SphereFunction

# create a sphere function instance with given dimensionality
objective_function = SphereFunction(DIM)

Let's also create an instance of a harder, multimodal objective function:

In [ ]:
from optilab.functions.multimodal import RastriginFunction

multimodal_objective = RastriginFunction(DIM)

We can see some information about a function in it's metadata:

In [ ]:
print(objective_function.get_metadata())
print(multimodal_objective.get_metadata())

## Creating an optimizer
Now let's create an instance of CMA-ES optimizer. Since optilab comes with implementations of some optimizers it's as easy as importing it from the library. Let's also create an instance of LMM-CMA-ES, which we will use to compare the two later on:

In [ ]:
# define constants for the population size and sigma0 for cmaes optimizers
POPSIZE = DIM * 2
SIGMA0 = 1

# import optimizers from optilab
from optilab.optimizers import CmaEs, LmmCmaEs

# create instances of the optimizers
cmaes_optimizer = CmaEs(POPSIZE, SIGMA0)

- [ ] print metadata
### optimize
- [ ] use optimize
- [ ] use .run_optimization
- [ ] show the optimizationRun structure
### plotting
- [ ] convergence
- [ ] ecdf
- [ ] box
### save and run
- [ ] save to pickle
- [ ] !python -m optilab pickle_name.pkl

In [ ]:

from optilab.data_classes import Bounds
from optilab.utils import dump_to_pickle

# hyperparams:
DIM = args.dim
POPSIZE = DIM * 2
NUM_NEIGHBORS = DIM + 2
BUFFER_SIZES = [m * POPSIZE for m in [2, 5, 10, 20, 30, 50]]
NUM_RUNS = 51
CALL_BUDGET = 1e4 * DIM
TOL = 1e-8
SIGMA0 = 1
NUM_PROCESSES = 16

# optimized problem
BOUNDS = Bounds(-100, 100)
TARGET = 0.0

for func in FUNCS[args.year]:
    print(func.name)
    results = []

    cmaes_optimizer = CmaEs(POPSIZE, SIGMA0)
    cmaes_results = cmaes_optimizer.run_optimization(
        NUM_RUNS, func, BOUNDS, CALL_BUDGET, TOL, num_processes=NUM_PROCESSES
    )
    results.append(cmaes_results)

    for buffer_size in BUFFER_SIZES:
        knn_optimizer = KnnCmaEs(POPSIZE, SIGMA0, NUM_NEIGHBORS, buffer_size)
        knn_results = knn_optimizer.run_optimization(
            NUM_RUNS, func, BOUNDS, CALL_BUDGET, TOL, num_processes=NUM_PROCESSES
        )
        results.append(knn_results)

    dump_to_pickle(
        results, f"003_knn_benchmark_{func.name}_{DIM}.pkl"
    )